07_merge_evictions_with_outcomes.ipynb

In [1]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
INPUT_DATA_EVICTIONS_PARCELS_AND_TRACTS = "./seniorthesis/data/02_intermediate/evictions_with_parcels_and_tracts.csv"
INPUT_DATA_TAX_PARCELS_EAST = "./seniorthesis/data/01_raw/Statewide_parcels_SHP/L3_TAXPAR_POLY_ASSESS_WEST.shp"
INPUT_DATA_TAX_PARCELS_WEST = "./seniorthesis/data/01_raw/Statewide_parcels_SHP/L3_TAXPAR_POLY_ASSESS_EAST.shp"
INPUT_DATA_ZESTIMATES = "./seniorthesis/data/02_intermediate/zestimates.csv"
INPUT_DATA_CRIME = "./seniorthesis/data/01_raw/crime_incidents"
OUTPUT_DATA_UNRESTRICTED = "./seniorthesis/data/03_cleaned/unrestricted.csv"
OUTPUT_DATA_ZILLOW = "./seniorthesis/data/03_cleaned/zestimates_analysis.csv"
VERBOSE = True

/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from Py

In [4]:
# Submit job to Oscar.
cluster = SLURMCluster(queue='batch',
                       cores=4,
                       memory='2 GB')
cluster.scale(jobs=8)
client =  Client(cluster)

/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46266 instead
  warnings.warn(
